# Mink usage examples

This notebook demonstrates some examples of using mink and how it interacts with sklearn.

## Imports

In [1]:
import pickle

In [2]:
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [3]:
from mink import NeuralNetClassifier
from mink import NeuralNetRegressor
from mink.layers import DenseLayer
from mink.layers import InputLayer
from mink.updates import SGD

## MNIST classification task

### Data

In [4]:
X, y = make_classification(n_samples=5000)

### Define neural net

Note: No need to specify the shape of the training data, number of classes, or to set softmax nonlinearity. The `NeuralNetClassifier` class takes care of all of that, as is expected from an sklearn estimator.

In [5]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [6]:
net = NeuralNetClassifier(l2, verbose=1)

If we want to change certain parameters after initialization, just use the `set_params` method and the double-underscore notation known from sklearn.

In [7]:
net.set_params(update__learning_rate=0.5)

NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7f03cf1059b0>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

### Train

In [8]:
net.fit(X, y, num_epochs=0)

# Neural Network with 4602 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |     20 |
|   1 | dense  |    200 |
|   2 | dense  |      2 |



NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7f03cf1059b0>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [9]:
y_proba = net.predict_proba(X)

In [10]:
(np.argmax(y_proba, 1) == y).mean()

0.59140000000000004

In [11]:
net.fit(X, y, num_epochs=20)

# Neural Network with 4602 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |     20 |
|   1 | dense  |    200 |
|   2 | dense  |      2 |

epochs:    1 | loss: 0.02135
epochs:    2 | loss: 0.01535
epochs:    3 | loss: 0.01394
epochs:    4 | loss: 0.01327
epochs:    5 | loss: 0.01313
epochs:    6 | loss: 0.01314
epochs:    7 | loss: 0.01307
epochs:    8 | loss: 0.01285
epochs:    9 | loss: 0.01210
epochs:   10 | loss: 0.01165
epochs:   11 | loss: 0.01199
epochs:   12 | loss: 0.01217
epochs:   13 | loss: 0.01246
epochs:   14 | loss: 0.01120
epochs:   15 | loss: 0.01112
epochs:   16 | loss: 0.01057
epochs:   17 | loss: 0.00976
epochs:   18 | loss: 0.00930
epochs:   19 | loss: 0.00996
epochs:   20 | loss: 0.00907


NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7f03cf1059b0>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [12]:
y_proba = net.predict_proba(X)

In [13]:
(np.argmax(y_proba, 1) == y.flatten()).mean()

0.93559999999999999

### Grid search

The neural net estimators can be used in conjunction with other sklearn features, such as `GridSearchCV`.

In [14]:
l0 = InputLayer()
l1 = DenseLayer(l0, name='hidden')
l2 = DenseLayer(l1)

In [15]:
net = NeuralNetClassifier(l2, update=SGD())

Parameters are set using the known double-underscore notation, e.g.

`'update__learning_rate': [0.1, 0.5]`.

Note: Instead of having to write

`'layer__incoming__num_units': [50, 100]`

we can just write

`'hidden__num_units': [50, 100]`

because we have given the hidden layer a name, "hidden". This may safe a lot of writing and confusion.

In [16]:
params = {
    'update__learning_rate': [0.1, 0.5],
    'max_epochs': [5, 10],
    'hidden__num_units': [50, 100],
}

In [17]:
cv = GridSearchCV(net, params, scoring='accuracy', refit=False, verbose=3)

In [18]:
cv.fit(X, y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] update__learning_rate=0.1, hidden__num_units=50, max_epochs=5 ...
[CV]  update__learning_rate=0.1, hidden__num_units=50, max_epochs=5, score=0.905219 -   0.3s
[CV] update__learning_rate=0.1, hidden__num_units=50, max_epochs=5 ...
[CV]  update__learning_rate=0.1, hidden__num_units=50, max_epochs=5, score=0.907618 -   0.3s
[CV] update__learning_rate=0.1, hidden__num_units=50, max_epochs=5 ...
[CV]  update__learning_rate=0.1, hidden__num_units=50, max_epochs=5, score=0.912365 -   0.4s
[CV] update__learning_rate=0.5, hidden__num_units=50, max_epochs=5 ...
[CV]  update__learning_rate=0.5, hidden__num_units=50, max_epochs=5, score=0.907618 -   0.4s
[CV] update__learning_rate=0.5, hidden__num_units=50, max_epochs=5 ...
[CV]  update__learning_rate=0.5, hidden__num_units=50, max_epochs=5, score=0.899220 -   0.5s
[CV] update__learning_rate=0.5, hidden__num_units=50, max_epochs=5 ...
[CV]  update__learning_rate=0.5, hidden__num_unit

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   18.4s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=Fa...ject at 0x7f03cf1059b0>,),
          session_kwargs=None, update=SGD(learning_rate=0.01), verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'update__learning_rate': [0.1, 0.5], 'hidden__num_units': [50, 100], 'max_epochs': [5, 10]},
       pre_dispatch='2*n_jobs', refit=False, scoring='accuracy', verbose=3)

In [19]:
cv.best_params_, cv.best_score_

({'hidden__num_units': 100, 'max_epochs': 10, 'update__learning_rate': 0.1},
 0.91400000000000003)

## Regression task

As is known from sklearn, we have separate estimators for classification and regression.

### Data

In [20]:
X, y = make_regression(n_samples=5000)

Note that apart from using `NeuralNetRegressor` instead of `NeuralNetClassifier`, everything is the same. No need to adjust output nonlinearity or objective.

### Define neural network

In [29]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [30]:
net = NeuralNetRegressor(l2, verbose=1)

In [31]:
net.set_params(update__learning_rate=0.0001)

NeuralNetRegressor(batch_iterator_test=128, batch_iterator_train=128,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=MeanSquaredError(),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7f03cf0b95c0>,),
          session_kwargs=None, update=SGD(learning_rate=0.0001), verbose=1)

### Train

In [32]:
net.fit(X, y, num_epochs=20)

# Neural Network with 20401 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |    100 |
|   1 | dense  |    200 |
|   2 | dense  |      1 |

epochs:    1 | loss: 42930.55469
epochs:    2 | loss: 35788.97266
epochs:    3 | loss: 9354.57910
epochs:    4 | loss: 557.85809
epochs:    5 | loss: 485.91931
epochs:    6 | loss: 199.16635
epochs:    7 | loss: 120.12661
epochs:    8 | loss: 90.05202
epochs:    9 | loss: 74.02477
epochs:   10 | loss: 63.88093
epochs:   11 | loss: 56.46662
epochs:   12 | loss: 50.66094
epochs:   13 | loss: 45.84669
epochs:   14 | loss: 41.69115
epochs:   15 | loss: 38.10532
epochs:   16 | loss: 34.70912
epochs:   17 | loss: 31.66856
epochs:   18 | loss: 28.80871
epochs:   19 | loss: 26.09939
epochs:   20 | loss: 23.70224


NeuralNetRegressor(batch_iterator_test=128, batch_iterator_train=128,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Linear(), num_units=1),
          max_epochs=10, objective=MeanSquaredError(),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7f03cf0b95c0>,),
          session_kwargs=None, update=SGD(learning_rate=0.0001), verbose=1)

## Saving and restoring

### Save previous net

In [33]:
score_before = mean_squared_error(y, net.predict(X))
print(score_before)

167.929876025


In [34]:
with open('mynet.pkl', 'wb') as f:
    pickle.dump(net, f)

### Create a new net with same architecture

In [35]:
with open('mynet.pkl', 'rb') as f:
    new_net = pickle.load(f)

In [36]:
score_after = mean_squared_error(y, new_net.predict(X))
print(score_after)
assert np.isclose(score_before, score_after)

167.929876025
